##### Copyright 2018 The TensorFlow Authors.

In [1]:
from flask import Flask, render_template, request, send_file
app = Flask(__name__, template_folder='template')
UPLOAD_FOLDER = 'static'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
from werkzeug import secure_filename

C:\Users\Monas\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: The import 'werkzeug.secure_filename' is deprecated and will be removed in Werkzeug 1.0. Use 'from werkzeug.utils import secure_filename' instead.
  


In [2]:
import os
import tensorflow as tf
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'
import tensorflow_hub as hub
hub_model = hub.load('Dataset')

In [3]:
import tensorflow_hub as hub
hub_model = hub.load('Dataset')

In [4]:
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

In [5]:
def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

In [ ]:
@app.route('/')
def index():
    return render_template('transfer.html')

@app.route('/transfer', methods = ['GET', 'POST'])
def transfer():
    if request.method == 'POST':
        f1 = request.files['img']
        pre = str(np.random.randint(5000))
        f1.save(secure_filename(f1.filename+pre))
        f2 = request.files['style']
        f2.save(secure_filename(f2.filename+pre))
        content_image = load_img(f1.filename)
        style_image = load_img(f2.filename)
        stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
        im1= tensor_to_image(stylized_image)
        fname= "styled"+pre+".png"
        im1.save(fname)
        return send_file(fname, attachment_filename='yourResult.png')
        

app.run('localhost')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Apr/2021 22:11:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 22:11:45] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [05/Apr/2021 22:12:32] "POST /transfer HTTP/1.1" 200 -


In [ ]:
app.run('localhost')